<a href="https://colab.research.google.com/github/Durixas/Parkinson-s-Disease-Prediction-Code-and-Data-Repository/blob/main/post_agg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
roc_auc_score, roc_curve, confusion_matrix, cohen_kappa_score, confusion_matrix, matthews_corrcoef

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_id = '1irvCs70iiI8WkXSZMMStmfPGfke5ivrP'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data.csv')

# Read the CSV file as DataFrame
import numpy as np
import pandas as pd

data = pd.read_csv('data.csv', header=1) # pd.read_excel() for .xlsx file

# Data Preprocessing

In [ ]:
data

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,250,0,0.80903,0.56355,0.28385,417,416,0.004627,0.000052,0.00064,...,3.0706,3.0190,3.1212,2.4921,3.5844,3.5400,3.3805,3.2003,6.8671,0
752,250,0,0.16084,0.56499,0.59194,415,413,0.004550,0.000220,0.00143,...,1.9704,1.7451,1.8277,2.4976,5.2981,4.2616,6.3042,10.9058,28.4170,0
753,251,0,0.88389,0.72335,0.46815,381,380,0.005069,0.000103,0.00076,...,51.5607,44.4641,26.1586,6.3076,2.8601,2.5361,3.5377,3.3545,5.0424,0
754,251,0,0.83782,0.74890,0.49823,340,339,0.005679,0.000055,0.00092,...,19.1607,12.8312,8.9434,2.2044,1.9496,1.9664,2.6801,2.8332,3.7131,0


In [ ]:
def split_data_by_id(data, test_size=0.3, random_state=None):
    unique_ids = data['id'].unique()

    train_ids, test_ids = train_test_split(unique_ids, test_size=test_size, random_state=random_state)

    train_data = data[data['id'].isin(train_ids)]
    test_data = data[data['id'].isin(test_ids)]

    return train_data, test_data

train_data, test_data = split_data_by_id(data, test_size=0.3, random_state=19260817)

groups = train_data.id

print("Train Data Shape:", train_data.shape)
print("Test Data Shape:", test_data.shape)

Train Data Shape: (528, 755)
Test Data Shape: (228, 755)


In [ ]:
X_train, y_train = train_data.drop(columns=['id']).iloc[:,:-1], train_data.iloc[:,-1]
X_test, y_test = test_data.drop(columns=['id']).iloc[:,:-1], test_data.iloc[:,-1]

In [ ]:
y_train.sum()/y_train.count(), y_test.sum()/y_test.count()

(0.7443181818181818, 0.75)

In [ ]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    index=X_train.index,
    columns=X_train.columns
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    index=X_test.index,
    columns=X_test.columns
)
X_train, X_test = X_train_scaled, X_test_scaled

In [ ]:
X_test.shape

(228, 753)

In [ ]:
train_data.iloc[:, 1:-1] = X_train

<ipython-input-12-872e2856be07>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3     -1.034692
4     -1.034692
5     -1.034692
6      0.966471
7      0.966471
         ...   
748    0.966471
749    0.966471
753   -1.034692
754   -1.034692
755   -1.034692
Name: gender, Length: 528, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_data.iloc[:, 1:-1] = X_train
<ipython-input-12-872e2856be07>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3     -1.426806
4     -0.870124
5     -0.966104
6     -0.044700
7     -0.083091
         ...   
748    0.128064
749    0.089672
753    0.521580
754    0.128064
755    0.128064
Name: numPulses, Length: 528, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_data.iloc[:, 1:-1] = X_train
<ipyt

In [ ]:
# Bag Over-sampling (BOS)

random_state = 42
np.random.seed(random_state)

synthetic_data = train_data.copy()
grouped = synthetic_data.groupby('id')
centers = grouped.mean().iloc[:, :-1]
classes = grouped.first()['class']
minority_class_ids = centers[classes == 0].index
majority_class_ids = centers[classes == 1].index


minority_centers = centers.loc[minority_class_ids]
distances = cdist(minority_centers.values, minority_centers.values, metric='euclidean')
np.fill_diagonal(distances, np.inf)

K = 2
while len(minority_class_ids) <= len(majority_class_ids) - K:
    selected_idx = np.random.choice(minority_centers.index)
    selected_idx_row = minority_centers.index.get_loc(selected_idx)
    neighbor_indices = distances[selected_idx_row].argsort()[:K]
    neighbor_ids = minority_centers.index[neighbor_indices]

    for neighbor_id in neighbor_ids:
        original_group = grouped.get_group(selected_idx)
        neighbor_group = grouped.get_group(neighbor_id)

        new_id = data.id.max() + len(synthetic_data["id"].unique()) + 1
        for i in range(3):
            original_sample = original_group.sample(n=1, random_state=random_state + i).iloc[:, 1:-1].values
            neighbor_sample = neighbor_group.sample(n=1, random_state=random_state + i).iloc[:, 1:-1].values

            z = np.random.uniform(0, 1)
            new_sample_features = original_sample + z * (neighbor_sample - original_sample)
            new_sample = new_sample_features.squeeze().tolist() + [0]

            new_sample = pd.DataFrame([new_sample], columns=synthetic_data.columns[1:])
            new_sample['id'] = new_id

            synthetic_data = pd.concat([synthetic_data, new_sample], ignore_index=True)

    minority_class_ids = synthetic_data[synthetic_data['class'] == 0]['id'].unique()
synthetic_data

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,1,-1.034692,-2.039604,1.326383,0.732020,-1.426806,-1.420297,2.487074,-0.271986,0.723698,...,-0.534526,-0.591047,-0.527734,-0.019885,-0.458638,-0.473263,-0.628013,-0.580870,-0.791258,1
1,1,-1.034692,-2.546592,1.342106,0.290970,-0.870124,-0.864839,1.015330,3.254926,1.180666,...,-0.480345,-0.525732,-0.497033,-0.413890,-0.267058,-0.056289,-0.413387,-0.664775,-0.730831,1
2,1,-1.034692,-1.451798,1.193739,1.170593,-0.966104,-0.998915,0.725819,3.293964,2.157630,...,-0.511379,-0.547785,-0.462185,-0.402729,-0.405712,-0.310192,-0.719285,-0.772427,-0.815807,1
3,2,0.966471,0.088764,-1.178858,0.398241,-0.044700,-0.041228,-0.169520,-0.379055,-0.052358,...,2.035839,1.468668,0.469646,-0.321369,-0.518807,-0.477139,-0.118335,-0.055901,-0.682254,1
4,2,0.966471,0.549806,-1.188292,0.160049,-0.083091,-0.079535,-0.124287,-0.338662,0.184004,...,-0.532807,-0.352074,-0.452041,-0.480233,-0.489824,-0.397333,-0.187240,0.239907,-0.762325,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,512,-1.034692,0.547693,-0.384340,-1.115247,0.258876,0.261681,-0.869832,0.235810,-0.397713,...,-0.436079,-0.568872,-0.584505,-0.540623,-0.536454,-0.606293,-0.782489,-0.825451,-0.819845,0
782,512,-1.034692,0.471544,-0.457140,-1.810553,0.958334,0.959602,-0.965974,-0.490747,-0.720101,...,-0.165566,-0.551522,-0.617440,-0.546463,-0.537583,-0.569430,-0.678059,-0.741119,-0.815825,0
783,513,-1.034692,0.505524,-0.414595,-1.093837,0.636142,0.638118,-0.748299,-0.466429,-0.596849,...,0.270789,0.188955,0.451138,0.533276,0.249117,-0.073988,-0.417109,-0.567401,-0.771939,0
784,513,-1.034692,0.607636,-0.029357,-1.107666,0.689952,0.691810,-0.787165,-0.468266,-0.607443,...,-0.045184,-0.212793,-0.245317,-0.334252,-0.475429,-0.574737,-0.752089,-0.753467,-0.803238,0


In [ ]:
X_train, y_train = synthetic_data.drop(columns=['id']).iloc[:,:-1], synthetic_data.iloc[:,-1]
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
groups = synthetic_data.id

# Model

In [ ]:
def print_grid_search(gs):
    print ("Best score: " + str(gs.best_score_))
    print ("Best parameters set:")
    best_parameters = gs.best_params_
    for param_name in sorted(best_parameters.keys()):
        print(param_name + ':' + str(best_parameters[param_name]))

def get_metrics3(model, mode = 'test'):
    if mode == 'train':
        X, y = X_train, y_train
    elif mode == 'test':
        X, y = X_test, y_test
    results = {}

    # post-mean
    y_pred_prob = model.predict_proba(X)[:, 1].reshape(-1, 3).mean(axis=1)

    # post-min
    # y_pred_prob = model.predict_proba(X)[:, 1].reshape(-1, 3).min(axis=1)

    # post-max
    # y_pred_prob = model.predict_proba(X)[:, 1].reshape(-1, 3).max(axis=1)

    y_pred = np.round(y_pred_prob).astype(int)
    y = stats.mode(y.to_numpy().reshape(-1, 3), axis=1)[0]

    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    specificity = tn / (tn + fp)

    results['Accuracy'] = accuracy_score(y, y_pred)
    results['Precision'] = precision_score(y, y_pred)
    results['Recall'] = recall_score(y, y_pred)
    results['Specificity'] = specificity
    results['F1'] = f1_score(y, y_pred)
    # results['kappa'] = cohen_kappa_score(y, y_pred)
    results['AUC'] = roc_auc_score(y, y_pred_prob)
    results['MCC'] = matthews_corrcoef(y, y_pred)
    return results

def get_results3(model):
    res1, res2 = get_metrics3(model, mode='train'), get_metrics3(model, mode='test')
    print(f'Train:\n{res1}')
    print(f'Test:\n{res2}')
    return res1, res2

def get_coma(model):
    X, y = X_test, y_test
    y_pred = model.predict(X)
    return confusion_matrix(y, y_pred)

## Logistic Regression

In [ ]:
params = {
    'C': [0.01, 0.1, 1.0],
    'penalty': ['l1', 'l2']
}
group_kfold = GroupKFold(n_splits=5)
grid_lr = GridSearchCV(LogisticRegression(solver='liblinear',random_state=42), params, cv=group_kfold)
grid_lr.fit(X_train, y_train, groups=groups)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=LogisticRegression(random_state=42, solver='liblinear'),
             param_grid={'C': [0.01, 0.1, 1.0], 'penalty': ['l1', 'l2']})

In [ ]:
print_grid_search(grid_lr)

Best score: 0.8972423802612483
Best parameters set:
C:1.0
penalty:l1


In [ ]:
best_lr = grid_lr.best_estimator_
res_lr_train3, res_lr_test3 = get_results3(best_lr)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.8947368421052632, 'Precision': 0.9016393442622951, 'Recall': 0.9649122807017544, 'Specificity': 0.6842105263157895, 'F1': 0.9322033898305084, 'AUC': 0.8928901200369345, 'MCC': 0.7062046123204375}


In [ ]:
df_train_result3 = pd.DataFrame(res_lr_train3, index=['Logistic Regression'])
df_test_result3 = pd.DataFrame(res_lr_test3, index=['Logistic Regression'])

## Decision Tree

In [ ]:
params = {
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
group_kfold = GroupKFold(n_splits=5)
grid_dt = GridSearchCV(DecisionTreeClassifier(random_state=42), params, cv=group_kfold)
grid_dt.fit(X_train, y_train, groups=groups)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [ ]:
print_grid_search(grid_dt)

Best score: 0.8412191582002903
Best parameters set:
criterion:gini
min_samples_leaf:4
min_samples_split:2


In [ ]:
best_dt = grid_dt.best_estimator_
res_dt_train3, res_dt_test3 = get_results3(best_dt)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.7105263157894737, 'Precision': 0.8181818181818182, 'Recall': 0.7894736842105263, 'Specificity': 0.47368421052631576, 'F1': 0.8035714285714286, 'AUC': 0.7008310249307479, 'MCC': 0.25482359571881275}


In [ ]:
df_train_result3 = pd.concat([df_train_result3, pd.DataFrame(res_dt_train3, index=['Decision Tree'])], axis=0)
df_test_result3 = pd.concat([df_test_result3, pd.DataFrame(res_dt_test3, index=['Decision Tree'])], axis=0)

## Random Forest

In [ ]:
params = {
    'n_estimators': [100, 200, 300],
    'max_features': ['log2', 'sqrt', 1]
}
group_kfold = GroupKFold(n_splits=5)
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), params, cv=group_kfold)
grid_rf.fit(X_train, y_train, groups=groups)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_features': ['log2', 'sqrt', 1],
                         'n_estimators': [100, 200, 300]})

In [ ]:
print_grid_search(grid_rf)

Best score: 0.9150217706821481
Best parameters set:
max_features:sqrt
n_estimators:200


In [ ]:
best_rf = grid_rf.best_estimator_
res_rf_train3, res_rf_test3 = get_results3(best_rf)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.8026315789473685, 'Precision': 0.85, 'Recall': 0.8947368421052632, 'Specificity': 0.5263157894736842, 'F1': 0.8717948717948718, 'AUC': 0.88134810710988, 'MCC': 0.4472135954999579}


In [ ]:
df_train_result3 = pd.concat([df_train_result3, pd.DataFrame(res_rf_train3, index=['Random Forest'])], axis=0)
df_test_result3 = pd.concat([df_test_result3, pd.DataFrame(res_rf_test3, index=['Random Forest'])], axis=0)

## Gradient Boosting

In [ ]:
params = {
    'n_estimators': [800],
    'learning_rate': [0.01, 0.02],
    'max_depth': [3, 5],
    'max_features': ['sqrt', 'log2']
}
group_kfold = GroupKFold(n_splits=5)
grid_gb = GridSearchCV(GradientBoostingClassifier(random_state=42), params, cv=group_kfold)
grid_gb.fit(X_train, y_train, groups=groups)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=GradientBoostingClassifier(random_state=42),
             param_grid={'learning_rate': [0.01, 0.02], 'max_depth': [3, 5],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [800]})

In [ ]:
print_grid_search(grid_gb)

Best score: 0.9250604741170779
Best parameters set:
learning_rate:0.02
max_depth:5
max_features:log2
n_estimators:800


In [ ]:
best_gb = grid_gb.best_estimator_
res_gb_train3, res_gb_test3 = get_results3(best_gb)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.8947368421052632, 'Precision': 0.8888888888888888, 'Recall': 0.9824561403508771, 'Specificity': 0.631578947368421, 'F1': 0.9333333333333333, 'AUC': 0.8504155124653738, 'MCC': 0.7060987881136028}


In [ ]:
df_train_result3 = pd.concat([df_train_result3, pd.DataFrame(res_gb_train3, index=['GBDT'])], axis=0)
df_test_result3 = pd.concat([df_test_result3, pd.DataFrame(res_gb_test3, index=['GBDT'])], axis=0)

## XGBoost

In [ ]:
params = {
    'n_estimators': [1000],
    'learning_rate': [0.01],
    'max_depth': [3],
    'min_child_weight': [1],
    'gamma': [0, 0.1],
    'subsample': [0.9],
    'colsample_bytree': [0.3],
}
group_kfold = GroupKFold(n_splits=5)
grid_xgb = GridSearchCV(xgb.XGBClassifier(random_state=42), params, cv=group_kfold)
grid_xgb.fit(X_train, y_train, groups=groups)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=Non...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             param_grid={'colsample_bytree': [0.3], 'gamma': [0, 0.1],
                         'learning_rate': [0.01], 'max_depth': [3],
                         'min_child_weight': [1], 'n_estimators': [1000],
                         'subsample': [0.9]})

In [ ]:
print_grid_search(grid_xgb)

Best score: 0.9073052733430093
Best parameters set:
colsample_bytree:0.3
gamma:0.1
learning_rate:0.01
max_depth:3
min_child_weight:1
n_estimators:1000
subsample:0.9


In [ ]:
best_xgb = grid_xgb.best_estimator_
res_xgb_train3, res_xgb_test3 = get_results3(best_xgb)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.9210526315789473, 'Precision': 0.9180327868852459, 'Recall': 0.9824561403508771, 'Specificity': 0.7368421052631579, 'F1': 0.9491525423728814, 'AUC': 0.8956602031394275, 'MCC': 0.7825510568956199}


In [ ]:
df_train_result3 = pd.concat([df_train_result3, pd.DataFrame(res_xgb_train3, index=['XGBoost'])], axis=0)
df_test_result3 = pd.concat([df_test_result3, pd.DataFrame(res_xgb_test3, index=['XGBoost'])], axis=0)

## LightGBM

In [ ]:
params = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.2, 0.3, 0.4],
    'max_depth': [3],
    'num_leaves': [10],
    'subsample': [0.3, 0.5],
    'colsample_bytree': [0.7, 0.8, 0.9]
}
group_kfold = GroupKFold(n_splits=5)
grid_lgb = GridSearchCV(lgb.LGBMClassifier(n_jobs=-1, verbose=-1), params, cv=group_kfold)
grid_lgb.fit(X_train, y_train, groups=groups)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=LGBMClassifier(n_jobs=-1, verbose=-1),
             param_grid={'colsample_bytree': [0.7, 0.8, 0.9],
                         'learning_rate': [0.2, 0.3, 0.4], 'max_depth': [3],
                         'n_estimators': [50, 100, 150], 'num_leaves': [10],
                         'subsample': [0.3, 0.5]})

In [ ]:
print_grid_search(grid_lgb)

Best score: 0.9275761973875183
Best parameters set:
colsample_bytree:0.9
learning_rate:0.3
max_depth:3
n_estimators:100
num_leaves:10
subsample:0.3


In [ ]:
best_lgb = grid_lgb.best_estimator_
res_lgb_train3, res_lgb_test3 = get_results3(best_lgb)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.8947368421052632, 'Precision': 0.8888888888888888, 'Recall': 0.9824561403508771, 'Specificity': 0.631578947368421, 'F1': 0.9333333333333333, 'AUC': 0.8910433979686057, 'MCC': 0.7060987881136028}


In [ ]:
df_train_result3 = pd.concat([df_train_result3, pd.DataFrame(res_lgb_train3, index=['LightGBM'])], axis=0)
df_test_result3 = pd.concat([df_test_result3, pd.DataFrame(res_lgb_test3, index=['LightGBM'])], axis=0)

## KNN

In [ ]:
params = {
    'n_neighbors': [2, 3, 4],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}
group_kfold = GroupKFold(n_splits=5)
grid_knn = GridSearchCV(KNeighborsClassifier(n_jobs=-1, algorithm='ball_tree'), params, cv=group_kfold)
grid_knn.fit(X_train, y_train, groups=groups)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=KNeighborsClassifier(algorithm='ball_tree', n_jobs=-1),
             param_grid={'n_neighbors': [2, 3, 4], 'p': [1, 2],
                         'weights': ['uniform', 'distance']})

In [ ]:
print_grid_search(grid_knn)

Best score: 0.8503144654088051
Best parameters set:
n_neighbors:2
p:1
weights:distance


In [ ]:
best_knn = grid_knn.best_estimator_
res_knn_train3, res_knn_test3 = get_results3(best_knn)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.75, 'Precision': 0.8653846153846154, 'Recall': 0.7894736842105263, 'Specificity': 0.631578947368421, 'F1': 0.8256880733944955, 'AUC': 0.7497691597414589, 'MCC': 0.3922322702763681}


In [ ]:
df_train_result3 = pd.concat([df_train_result3, pd.DataFrame(res_knn_train3, index=['KNN'])], axis=0)
df_test_result3 = pd.concat([df_test_result3, pd.DataFrame(res_knn_test3, index=['KNN'])], axis=0)

## SVM

In [ ]:
params = {
    'C': [0.5, 1, 2],
    'gamma': ['scale', 'auto'],
}
group_kfold = GroupKFold(n_splits=5)
grid_svm = GridSearchCV(SVC(random_state=42, probability=True), params, cv=group_kfold)
grid_svm.fit(X_train, y_train, groups=groups)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=SVC(probability=True, random_state=42),
             param_grid={'C': [0.5, 1, 2], 'gamma': ['scale', 'auto']})

In [ ]:
print_grid_search(grid_svm)

Best score: 0.916231253023706
Best parameters set:
C:2
gamma:scale


In [ ]:
best_svm = grid_svm.best_estimator_
res_svm_train3, res_svm_test3 = get_results3(best_svm)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.868421052631579, 'Precision': 0.8615384615384616, 'Recall': 0.9824561403508771, 'Specificity': 0.5263157894736842, 'F1': 0.9180327868852459, 'AUC': 0.8199445983379502, 'MCC': 0.6261585997769192}


In [ ]:
df_train_result3 = pd.concat([df_train_result3, pd.DataFrame(res_svm_train3, index=['SVM'])], axis=0)
df_test_result3 = pd.concat([df_test_result3, pd.DataFrame(res_svm_test3, index=['SVM'])], axis=0)

## Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)

GaussianNB()

In [ ]:
res_nb_train3, res_nb_test3 = get_results3(nb)

Train:
{'Accuracy': 0.8053435114503816, 'Precision': 0.8389830508474576, 'Recall': 0.7557251908396947, 'Specificity': 0.8549618320610687, 'F1': 0.7951807228915663, 'AUC': 0.8831361808752403, 'MCC': 0.6137164119322156}
Test:
{'Accuracy': 0.7105263157894737, 'Precision': 0.8888888888888888, 'Recall': 0.7017543859649122, 'Specificity': 0.7368421052631579, 'F1': 0.7843137254901961, 'AUC': 0.7807017543859649, 'MCC': 0.38644907980014026}


In [ ]:
df_train_result3 = pd.concat([df_train_result3, pd.DataFrame(res_nb_train3, index=['Naive Bayes'])], axis=0)
df_test_result3 = pd.concat([df_test_result3, pd.DataFrame(res_nb_test3, index=['Naive Bayes'])], axis=0)

## AdaBoost

In [ ]:
best_ada = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100,
    learning_rate=0.5,
    random_state=42
).fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [ ]:
res_ada_train3, res_ada_test3 = get_results3(best_ada)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.9078947368421053, 'Precision': 0.9032258064516129, 'Recall': 0.9824561403508771, 'Specificity': 0.6842105263157895, 'F1': 0.9411764705882353, 'AUC': 0.8550323176361958, 'MCC': 0.7446687115861395}


In [ ]:
df_train_result3 = pd.concat([df_train_result3, pd.DataFrame(res_ada_train3, index=['AdaBoost'])], axis=0)
df_test_result3 = pd.concat([df_test_result3, pd.DataFrame(res_ada_test3, index=['AdaBoost'])], axis=0)

## MLP

In [ ]:
mlp_model = MLPClassifier(random_state=42, hidden_layer_sizes=(512,256,128), learning_rate_init=0.001)
mlp_model.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(512, 256, 128), random_state=42)

In [ ]:
res_mlp_train3, res_mlp_test3 = get_results3(mlp_model)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.9078947368421053, 'Precision': 0.9032258064516129, 'Recall': 0.9824561403508771, 'Specificity': 0.6842105263157895, 'F1': 0.9411764705882353, 'AUC': 0.8679593721144968, 'MCC': 0.7446687115861395}


In [ ]:
df_train_result3 = pd.concat([df_train_result3, pd.DataFrame(res_mlp_train3, index=['MLP'])], axis=0)
df_test_result3 = pd.concat([df_test_result3, pd.DataFrame(res_mlp_test3, index=['MLP'])], axis=0)

## Stacking

In [ ]:
estimators = [
    ('knn', KNeighborsClassifier(n_jobs=-1, algorithm='ball_tree')),
    ('svc', SVC(probability=True, gamma='auto', random_state=42))
]
final_estimator = LogisticRegression(solver='liblinear', random_state=42)

stacking_clf = StackingClassifier(estimators=estimators, final_estimator=final_estimator)

params = {
    'knn__n_neighbors': [2, 3, 4],
    'svc__C': [0.1, 1, 10],
    'final_estimator__C': [1, 10],
    'final_estimator__penalty': ['l1', 'l2']
}
group_kfold = GroupKFold(n_splits=5)
grid_stack = GridSearchCV(estimator=stacking_clf, param_grid=params, cv=group_kfold, n_jobs=-1)
grid_stack.fit(X_train, y_train, groups=groups)

GridSearchCV(cv=GroupKFold(n_splits=5),
             estimator=StackingClassifier(estimators=[('knn',
                                                       KNeighborsClassifier(algorithm='ball_tree',
                                                                            n_jobs=-1)),
                                                      ('svc',
                                                       SVC(gamma='auto',
                                                           probability=True,
                                                           random_state=42))],
                                          final_estimator=LogisticRegression(random_state=42,
                                                                             solver='liblinear')),
             n_jobs=-1,
             param_grid={'final_estimator__C': [1, 10],
                         'final_estimator__penalty': ['l1', 'l2'],
                         'knn__n_neighbors': [2, 3, 4],
                         'svc__C': [0.1, 1, 10]})

In [ ]:
print_grid_search(grid_stack)

Best score: 0.9288582486695696
Best parameters set:
final_estimator__C:1
final_estimator__penalty:l1
knn__n_neighbors:2
svc__C:10


In [ ]:
best_stack = grid_stack.best_estimator_
res_stack_train3, res_stack_test3 = get_results3(best_stack)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.881578947368421, 'Precision': 0.875, 'Recall': 0.9824561403508771, 'Specificity': 0.5789473684210527, 'F1': 0.9256198347107438, 'AUC': 0.840258541089566, 'MCC': 0.6666666666666666}


In [ ]:
df_train_result3 = pd.concat([df_train_result3, pd.DataFrame(res_stack_train3, index=['Stacking'])], axis=0)
df_test_result3 = pd.concat([df_test_result3, pd.DataFrame(res_stack_test3, index=['Stacking'])], axis=0)

# Evaluation

In [ ]:
df_test_result_sorted3 = df_test_result3.sort_values(by=['Accuracy', 'MCC', 'AUC'], ascending=False)
df_train_result_reordered3 = df_train_result3.reindex(df_test_result_sorted3.index)
print('Train:')
df_train_result_reordered3

Train:


,Accuracy,Precision,Recall,Specificity,F1,AUC,MCC
XGBoost,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
MLP,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
AdaBoost,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Logistic Regression,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
LightGBM,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
GBDT,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Stacking,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
SVM,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Random Forest,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
KNN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
print('Test:')
df_test_result_sorted3

Test:


,Accuracy,Precision,Recall,Specificity,F1,AUC,MCC
XGBoost,0.921053,0.918033,0.982456,0.736842,0.949153,0.895660,0.782551
MLP,0.907895,0.903226,0.982456,0.684211,0.941176,0.867959,0.744669
AdaBoost,0.907895,0.903226,0.982456,0.684211,0.941176,0.855032,0.744669
Logistic Regression,0.894737,0.901639,0.964912,0.684211,0.932203,0.892890,0.706205
LightGBM,0.894737,0.888889,0.982456,0.631579,0.933333,0.891043,0.706099
GBDT,0.894737,0.888889,0.982456,0.631579,0.933333,0.850416,0.706099
Stacking,0.881579,0.875000,0.982456,0.578947,0.925620,0.840259,0.666667
SVM,0.868421,0.861538,0.982456,0.526316,0.918033,0.819945,0.626159
Random Forest,0.802632,0.850000,0.894737,0.526316,0.871795,0.881348,0.447214
KNN,0.750000,0.865385,0.789474,0.631579,0.825688,0.749769,0.392232


In [ ]:
df_train_result_reordered3.to_csv('post_mean.csv', index=True)
df_test_result_sorted3.to_csv('post_mean.csv', index=True)